### define environment

In [ ]:
USING_GOOGLE_COLAB = False

In [ ]:
if USING_GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if USING_GOOGLE_COLAB:
    !git clone https://github.com/GrzegorzKazana/nearest-neighbours.git
    !ls drive

In [ ]:
import os
import sys

proj_base_path = '/content/nearest-neighbours' if USING_GOOGLE_COLAB else ''
data_base_path = '/content/drive/My Drive/' if USING_GOOGLE_COLAB else ''
models_base_path = '/content/drive/My Drive/' if USING_GOOGLE_COLAB else ''
sys.path.append(os.path.join(os.getcwd(), proj_base_path))

### code starts here

In [ ]:
import numpy as np

images_file_path = ''
labels_file_path = ''

images = np.load(os.path.join(data_base_path, images_file_path))
labels = np.load(os.path.join(data_base_path, labels_file_path))

print(images.shape, labels.shape)
print(f'number of samples {len(images)}')

assert len(images) == len(labels)

n_of_samples = len(images)

In [ ]:
def data_generator(X, Y, batch_size):
    while True:
        batch_idxs = np.random.randint(0, len(X), size=batch_size)

        x = X[batch_idxs]
        y = Y[batch_idxs]

        yield x, y
        
data_gen = data_generator(images, labels, 16)
next(data_gen)

### model preparation

In [ ]:
INPUT_SHAPE = (224, 224, 3)
HIDDEN_SIZE = 1024
OUTPUT_SIZE = 50

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

input_layer = Input(shape=INPUT_SHAPE)
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
output_vgg16_conv = model_vgg16_conv(input_layer)

x = Flatten()(output_vgg16_conv)
x = Dense(HIDDEN_SIZE, activation='relu')(x)
x = Dense(OUTPUT_SIZE, activation='softmax')(x)

model = Model(input_layer, outputs=x)

model.summary()

### training

In [ ]:
BATCH_SIZE = 16
data_gen = data_generator(images, labels, BATCH_SIZE)

In [ ]:
STEPS_PER_EPOCH = 100
EPOCHS = 10

model.fit_generator(
    data_gen,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=EPOCHS
)

In [ ]:
from datetime import datetime

t = datetime.timestamp(datetime.now())
models_file_name = f'm_{t}.h5'

model.save(os.path.join(models_base_path, models_file_name))